In [10]:
import numpy as np
import pandas as pd 
import time
import cv2
import os
from imutils import paths
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import cv2
import shutil
import xml.etree.ElementTree as ET
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input
import tensorflow.keras.layers as tfl
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split


####################
# Helper Functions #
####################
#locates h5 file in the h5_to_uff folder
def find_h5_file(folder):
    for filepath in glob.glob(folder + '\*'):
        filename = filepath.split('\\')[-1]
        if '.h5' in filename:
            return filename

def xml_gen(file,path,b_width,b_height):
        f = open(( file.rsplit( ".", 1 )[ 0 ] ) + ".xml", "w") #creates a new file using the .jpg filename, but with the .fsv extension
        f.write("<annotation>") #write to the text file
        f.write("\n")  
        f.write("    <folder>" + os.path.basename(path) + "</folder>")  
        f.write("\n")        
        f.write("    <filename>" + file + "</filename>")    
        f.write("\n")        
        f.write("    <path>" + os.path.join(path,file) + "</path>")     
        f.write("\n")        
        f.write("    <source>\n     <database>Unknown</database>\n    </source>")   
        f.write("\n")  
        f.write("    <size>") 
        f.write("\n") 
        f.write("        <width>" + str(b_width) + "</width>")     
        f.write("\n")
        f.write("        <height>" + str(b_height) + "</height>")
        f.write("\n")  
        f.write("        <depth>3</depth>") 
        f.write("\n")  
        f.write("    </size>") 
        f.write("\n")        
        f.write("    <segmented>0</segmented>")     
        f.write("\n")
        

def obj_xml(file,label,xmin,ymin,xmax,ymax):
        f = open(( file.rsplit( ".", 1 )[ 0 ] ) + ".xml", "a") #creates a new file using the .jpg filename, but with the .fsv extension
        f.write("    <object>")
        f.write("\n")
        f.write("        <name>" + label + "</name>")
        f.write("\n")       
        f.write("        <pose>Unspecified</pose>\n        <truncated>0</truncated>\
        \n        <difficult>0</difficult>") 
        f.write("\n")        
        f.write("        <bndbox>")
        f.write("\n")   
        f.write("            <xmin>" + str(xmin) + "</xmin>") 
        f.write("\n")   
        f.write("            <ymin>" + str(ymin) + "</ymin>")   
        f.write("\n")        
        f.write("            <xmax>" + str(xmax) + "</xmax>")     
        f.write("\n")
        f.write("            <ymax>" + str(ymax) + "</ymax>")
        f.write("\n") 
        f.write("    </bndbox>")
        f.write("\n") 
        f.write("    </object>")
        f.write("\n") 


def close_xml(file):
        f = open(( file.rsplit( ".", 1 )[ 0 ] ) + ".xml", "a") #creates a new file using the .jpg filename, but with the .fsv extension
        f.write("</annotation>") 
        f.close()


def find_xml(fname, DATA_DIR):
    xml_name = fname[:-3]+'xml'
    xml_path = DATA_DIR+'/'+xml_name
    # print(xml_path)
    return xml_path


def extract_coords(xml_path):
    x_value = 1
    y_value = 1
    coords = []
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for object in root.findall('object'):
        name = object.find('name').text
        if(name == 'person'):
            bbox = object.find('bndbox')
            # for cords in bbox:
            xmin = int(int((bbox.find('xmin').text))/x_value) #1.02 - 1.03 for side views
            ymin = int(int((bbox.find('ymin').text))/y_value)
            xmax = int(int((bbox.find('xmax').text))*x_value)
            ymax = int(int((bbox.find('ymax').text))*y_value)
            coords.append([xmin,ymin,xmax,ymax])
    return coords

#creates class folders in a save directory
def create_class_folders(save_dir, classes):
    for clss in classes:
        try:
            os.mkdir(save_dir + '\\' + clss)
        except:
            print('unable to create class folder')

In [13]:
###################
# Load Classifier #
###################
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

#classifier confidence setting
c_confidence = 0.60

model_version = 47
weights_version = find_h5_file(r'D:\Daniel\PMD\scripts\H5 to UFF\pmd_v{}'.format(model_version))

print(f"Loading Model v{model_version}")

model_path = r"D:\Daniel\PMD\scripts\H5 to UFF\pmd_v{}\{}".format(model_version, weights_version)
model = load_model(model_path)

print('Model Loaded')
classes = ["cyclist", "ebike", 'motorcycle', 'pedestrian', 'standing_scooter']

Loading Model v47
Model Loaded


In [8]:
########################
# Load Person Detector #
########################
configPath = r'D:\Daniel\PMD\scripts\yoloV4PA\Yolov4-custom_new.cfg'
labelsPath = r'D:\Daniel\PMD\scripts\yoloV4PA\YoloV4PA.names'
weightsPath = r'D:\Daniel\PMD\scripts\yoloV4PA\yoloV4PA.weights'

LABELS = open(labelsPath).read().strip().split("\n")


print(weightsPath)
# load our YOLO object detector trained on OCR dataset (34 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

    # determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

D:\Daniel\PMD\scripts\yoloV4PA\yoloV4PA.weights
[INFO] loading YOLO from disk...


In [7]:
#################################
# Test PMD Classifier on images #
#################################
IMG_SIZE = (260,110)
data_dir = r'I:\Daniel\Israel Footage\pmd_labeller\images_to_test_pmd_labeller\pmd_data'

for folder in glob.glob(data_dir + '\*'):
    foldername = folder.split('\\')[-1]

    if foldername == 'finished':
        continue

    for fname in tqdm(os.listdir(folder)):
        try:
            img = cv2.imread(os.path.join(folder, fname))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_array = cv2.resize(img, (IMG_SIZE[1], IMG_SIZE[0]), cv2.INTER_LANCZOS4) #classifier trained on (260,110) images
            img_array = img_array/255
            img_array = np.expand_dims(img_array, axis=0) #expand dimensions from 3 to 4 because tensorflow model takes in batch size as 1 additional dim

            prediction = model.predict(img_array)
            pred_class_index = np.argmax(prediction, axis = 1)
            pred_class = classes[pred_class_index[0]]

            print(pred_class, fname)
        except Exception as e:
            print(e)

 67%|██████▋   | 2/3 [00:00<00:00,  6.80it/s]

cyclist 1631238740513.JPEG
motorcycle processed (1).jpeg


100%|██████████| 3/3 [00:00<00:00,  6.62it/s]

cyclist processed.jpeg


In [14]:
#################################
# Test PMD Classifier on videos #
#################################
frames_folder = r"I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames"

save_dir = r"I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\results"

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

#detector confidence and threshold settings
d_confidence = 0.50
d_threshold = 0.90

#classifier confidence setting
c_confidence = 0.50



# Step 1: Extract Frames
main_dir = os.getcwd()
os.chdir(r'D:\Daniel\PMD\scripts\yoloV4PA\ffmpeg\bin') #change directory to .exe script
cmd = r'python automate_cmd_ffmpeg.py'

# os.system(cmd)

# Step 2: Detect persons in frames
for video_folder in glob.glob(frames_folder + '\*'):

    if 'data_frames' not in video_folder: 

        imagePaths = sorted(list(paths.list_images(f"{video_folder}")))

        image_name = []

        plate_name = []

        for img in imagePaths:
            # load our input image and grab its spatial dimensions
            groundtruth = img.split('.jpg')[0]
            image = cv2.imread(img)
            (H, W) = image.shape[:2]

            #print(args["image"])


            # construct a blob from the input image and then perform a forward
            # pass of the YOLO object detector, giving us our bounding boxes and
            # associated probabilities
            temp_xml_file = groundtruth + ".xml"
            xml_gen(temp_xml_file,groundtruth,W,H) 
            blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (608, 608),swapRB=True, crop=False)
            net.setInput(blob)
            start = time.time()
            layerOutputs = net.forward(ln)
            end = time.time()

            # show timing information on YOLO
            print(groundtruth, "YOLO took {:.6f} seconds".format(end - start))

            # initialize our lists of detected bounding boxes, confidences, and
            # class IDs, respectively
            boxes = []
            confidences = []
            classIDs = []

            # loop over each of the layer outputs
            for output in layerOutputs:
                # loop over each of the detections
                for detection in output:
                    # extract the class ID and confidence (i.e., probability) of
                    # the current object detection
                    scores = detection[5:]
                    classID = np.argmax(scores)
                    confidence = scores[classID]

                    # filter out weak predictions by ensuring the detected
                    # probability is greater than the minimum probability
                    if confidence > d_confidence:
                        # scale the bounding box coordinates back relative to the
                        # size of the image, keeping in mind that YOLO actually
                        # returns the center (x, y)-coordinates of the bounding
                        # box followed by the boxes' width and height
                        box = detection[0:4] * np.array([W, H, W, H])
                        (centerX, centerY, width, height) = box.astype("int")

                        # use the center (x, y)-coordinates to derive the top and
                        # and left corner of the bounding box
                        x = int(centerX - (width / 2))
                        y = int(centerY - (height / 2))

                        # update our list of bounding box coordinates, confidences,
                        # and class IDs
                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))
                        classIDs.append(classID)
                        
            # apply non-maxima suppression to suppress weak, overlapping bounding
            # boxes
            idxs = cv2.dnn.NMSBoxes(boxes, confidences, d_confidence,d_threshold)
            prediction_img = []
            rect_bbox = []
            # prediction_img = LABELS[classID]
            # ensure at least one detection exists
            if len(idxs) > 0:
                # loop over the indexes we are keeping
                for i in idxs.flatten():
                    # extract the bounding box coordinates
                    (x, y) = (boxes[i][0], boxes[i][1])
                    (w, h) = (boxes[i][2], boxes[i][3])
                    prediction_img = LABELS[classIDs[i]]
                    rect_bbox.append([x,y,x+w,y+h,w,h,prediction_img])
                    # prediction_img.append(LABELS[classIDs[i]])
                    
            if len(rect_bbox)>0:
                
                for index_j, j in enumerate(rect_bbox):
                        for index_k, k in enumerate(rect_bbox):

                            if j!=k:
                                j_bbox = j
                                k_bbox = k
                                if j_bbox[2] <= k_bbox[0] or j_bbox[3] <= k_bbox[1] or j_bbox[0] >= k_bbox[2] or  j_bbox[1] >= k_bbox[3]:
                                    overlap = True
                                else:
                                    overlap = True
                                    if 'j_bbox' in globals() is True and 'k_bbox' in globals() is True:
                                        if j_bbox[6] == k_bbox[6] and j_bbox[2]<k_bbox[2]:
                                            rect_bbox.remove(k_bbox)
                                        elif j_bbox[6] == k_bbox[6] and j_bbox[2]>k_bbox[2]:
                                            rect_bbox.remove(j_bbox)


            for bbox in rect_bbox:
                    xmin = bbox[0] 
                    ymin = bbox[1] 
                    ymax = bbox[3] 
                    xmax = bbox[2]  
                    c = bbox[6]
                    if c =='person':
                        obj_xml(temp_xml_file, c,xmin,ymin,xmax,ymax)

                    
                    
            close_xml(temp_xml_file)


#Step 3: Classify and save cropped images
xmin_cropsize = 1.01
xmax_cropsize = 1.01
ymin_cropsize = 1.07
ymax_cropsize = 1.07
for videofolder in glob.glob(frames_folder + '\*'):
    save = save_dir + '\\' + foldername
    if os.path.exists(save):
        shutil.rmtree(save)
    os.mkdir(save)
    create_class_folders(save, classes)

    for fname in tqdm(os.listdir(videofolder)):
        if(r'.jpg' in fname):
            try:
                img = cv2.imread(os.path.join(videofolder, fname))
                #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                xml_path = find_xml(fname, videofolder)
                coords = extract_coords(xml_path)

                for (xmin, ymin, xmax, ymax) in coords:
                    if(xmax-xmin > 1 and ymax-ymin > 1):
                        xmin_new = int(xmin/xmin_cropsize)
                        xmax_new = int(xmax*xmax_cropsize)
                        ymin_new = int(ymin/ymin_cropsize)
                        ymax_new = int(ymax*ymax_cropsize)

                        width = xmax_new - xmin_new
                        height = ymax_new - ymin_new
                        if width>50 and height>50: #ensures that the person detected is not too small
                            
                            #PREDICT CLASS with classifier: "cyclist", "ebike", 'motorcycle', 'pedestrian','seated_scooter', 'standing_scooter'
                            cropped_img = img[ymin_new:ymax_new, xmin_new:xmax_new]
                            cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
                            cropped_img_array = cv2.resize(cropped_img, (IMG_SIZE[1], IMG_SIZE[0]), cv2.INTER_LANCZOS4) #classifier trained on (260,110) images
                            cropped_img_array = cropped_img_array/255
                            cropped_img_array = np.expand_dims(cropped_img_array, axis=0) #expand dimensions from 3 to 4 because tensorflow model takes in batch size as 1 additional dim

                            prediction = model.predict(cropped_img_array)
                            pred_class_index = np.argmax(prediction, axis = 1)
                            pred_class = classes[pred_class_index[0]]

                        #skip predictions that are below confidence value
                            if prediction[0][np.argmax(prediction, axis=1)[0]] < c_confidence:
                                continue

                            #############################
                            #     save cropped image    #
                            #############################
                            save_crop = save + '\\' + pred_class 
                            cv2.imwrite(os.path.join(save_crop, foldername +'_'+ fname[:-4]+'-'+str(len(os.listdir(save_dir)))+ ".jpg"), cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR))

            except Exception as e:
                print(str(e)) #prints the error
                pass    


I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0001 YOLO took 0.658652 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0002 YOLO took 0.617139 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0003 YOLO took 0.642144 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0004 YOLO took 0.676435 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0005 YOLO took 0.668150 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0006 YOLO took 0.730669 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0007 YOLO took 0.743167 seconds
I:\Daniel\Israel Footage\pmd_labeller\videos_to_test_pmd_labeller\extracted_frames\STE_PMD\frame0008 YOL

 43%|████▎     | 1403/3228 [21:51<31:54,  1.05s/it]

OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wef8acrf\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



 44%|████▎     | 1407/3228 [21:52<20:03,  1.51it/s]

OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wef8acrf\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



...

 60%|█████▉    | 1933/3228 [30:37<11:48,  1.83it/s]

OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wef8acrf\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



 97%|█████████▋| 3141/3228 [1:04:05<02:15,  1.56s/it]

OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wef8acrf\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



100%|██████████| 3228/3228 [1:06:31<00:00,  1.24s/it]


In [22]:
###################################################################################################
# Temporary Baby Stroller Classifier: to extract baby stroller images from existing training data #
###################################################################################################

main_dir = r'D:\Daniel\PMD\model_training\temporary_baby_stroller_classifier'
data_dir = main_dir + '\data'
train_dir = main_dir + '\\train'
valid_dir = main_dir + '\\valid'

# Prepare train-test sets
width = 110
height = 260

def create_folder(train_folder, valid_folder):
    if os.path.exists(train_folder):
        shutil.rmtree(train_folder)
    if os.path.exists(valid_folder):
        shutil.rmtree(valid_folder)

    os.mkdir(train_folder)
    os.mkdir(valid_folder)


#function to copy images into train/valid folder
def transfer_img(train_folder, valid_folder, train_dir, valid_dir):
    for imagepath in train_folder:
        try:
            shutil.copy(imagepath, train_dir)
        except:
            pass
    
    for imagepath in valid_folder:
        try:
            shutil.copy(imagepath, valid_dir)
        except:
            pass

# Prepare train-test sets
# img_lst = []
# create_folder(train_dir, valid_dir)
# for folder in glob.glob(data_dir + '\*'): #iterate through each class subfolder
#     classname = folder.split('\\')[-1] 
#     print(f'running {classname} folder')
   
#     for imagepath in glob.glob(folder + '\*'):
#         img_lst.append(imagepath)
#     train, valid = train_test_split(img_lst, train_size=0.95 , test_size = 0.05)
#     train_save = train_dir + '\\' + classname
#     valid_save = valid_dir + '\\' + classname 
#     os.mkdir(train_save)
#     os.mkdir(valid_save)
#     transfer_img(train, valid, train_save, valid_save)
#     img_lst = []


train_datagen = ImageDataGenerator(rescale = 1./255,
                                #    rotation_range = 40,
                                   zoom_range = 0.2,
                                   width_shift_range=0.2,
                                #    height_shift_range=0.2,
                                   horizontal_flip = True,
                                   brightness_range = [0.8, 1.0],
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (height, width),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory(valid_dir,
                                            target_size = (height, width),
                                            batch_size = 32,
                                            class_mode = 'binary')

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# Model

model = tf.keras.Sequential([
        tfl.Conv2D(128, 3, strides=1, input_shape=(height, width,3)),
        tfl.ReLU(),
        tfl.MaxPool2D(),
        tfl.Conv2D(64, 3, strides=1),
        tfl.ReLU(),
        tfl.MaxPool2D(),
        tfl.Conv2D(64, 3, strides=1),
        tfl.ReLU(),
        tfl.MaxPool2D(),
        tfl.Conv2D(64, 3, strides=1),
        tfl.BatchNormalization(axis=3),
        tfl.ReLU(),
        tfl.MaxPool2D(),
        tfl.Flatten(),
        tfl.Dense(64),
        tfl.ReLU(),
        tfl.Dropout(0.5),
        tfl.Dense(1, activation='sigmoid')
        ])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

reduce_lr_on_plateau = ReduceLROnPlateau(
                                        monitor='val_loss', 
                                        factor=0.1, 
                                        patience=5, 
                                        verbose=1,
                                        mode='min', 
                                        min_delta=0.0001, 
                                        cooldown=0, 
                                        min_lr=0,
)

history = model.fit(training_set, batch_size = 32, epochs=100, validation_data = test_set, callbacks = [reduce_lr_on_plateau])

Found 316 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Epoch 1/100
10/10 [==============================] - 5s 331ms/step - loss: 1.2663 - accuracy: 0.5348 - val_loss: 0.6907 - val_accuracy: 0.5000
Epoch 2/100
10/10 [==============================] - 2s 178ms/step - loss: 0.6481 - accuracy: 0.6234 - val_loss: 0.6890 - val_accuracy: 0.5000
Epoch 3/100
10/10 [==============================] - 2s 182ms/step - loss: 0.5667 - accuracy: 0.7025 - val_loss: 0.6866 - val_accuracy: 0.7778
Epoch 4/100
10/10 [==============================] - 2s 179ms/step - loss: 0.5041 - accuracy: 0.7848 - val_loss: 0.6839 - val_accuracy: 0.8333
Epoch 5/100
10/10 [==============================] - 2s 178ms/step - loss: 0.4937 - accuracy: 0.7437 - val_loss: 0.6839 - val_accuracy: 0.8333
Epoch 6/100
10/10 [==============================] - 2s 181ms/step - loss: 0.3872 - accuracy: 0.8038 - val_loss: 0.6559 - val_accuracy: 0.4444
Epoch 7/100
10/10 [==============================] - 2s 179ms

In [16]:
model.save('temp_baby_stroller_classifier.h5')

In [21]:
# Load Temp Baby Stroller Model
model = load_model('temp_baby_stroller_classifier.h5')

data_dir = r'D:\Daniel\PMD\model_training\final_data\combined_data_30.8.2021\train\pedestrian'
data_dir = r"D:\Daniel\PMD\model_training\temporary_baby_stroller_classifier\data_test"
save_dir = r"D:\Daniel\PMD\model_training\temporary_baby_stroller_classifier\save\baby_stroller"
classes = ['baby_stroller', 'not_baby_stroller']

width = 110
height = 260
c_confidence = 0.7
for imagepath in glob.glob(data_dir + '\*'):
    img_name = imagepath.split('\\')[-1]
    img = cv2.imread(imagepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = cv2.resize(img, (width, height), cv2.INTER_LANCZOS4) #classifier trained on (260,110) images
    img_array = img_array/255
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    pred_class_index = np.argmax(prediction, axis = 1)
    pred_class = classes[pred_class_index[0]]

    if prediction[0][np.argmax(prediction, axis=1)[0]] < c_confidence:
        continue
    
    if pred_class == 'baby_stroller':
        cv2.imwrite(os.path.join(save_dir, img_name), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))



In [ ]:
############################################
# Testing out detector on skateboard class #
############################################

import yolov5
import torch

model =  torch.hub.load('ultralytics/yolov5', 'yolov5s')

img = r"D:\Daniel\PMD\model_training\final_data\test_data\skateboard\frames\frame1654.jpg"  # or file, Path, PIL, OpenCV, numpy, list

# Inference
results = model(img)

# Results
results.print()
results.show() #.show() to display bounding boxes on image

[98 24  4 70 48  1 52 64 78 22]
